# First Stage Thrust Curves

The reference I can find with thrust curves for the Lambda vehicle has a total thrust curve for the entire first stage, the main motor and both boosters combined. We would like to know what the indavidual thrusts are.

We can probably make a pretty good guess to what that is, interpolating the thrust of the first stage motor by itself (it should be similar to the second stage) and then subtracting from the total.

## First Stage Total Thrust

In [ ]:
# dependencies
from numpy import loadtxt
import matplotlib.pyplot as plt
%matplotlib inline

g_0 = 9.80665

LB2KG = 0.453592
IN2M = 0.0254
SF2KGM = 1.35581795

stage1_thrust_time, stage1_thrust = loadtxt('../thrust_booster_and_stage1.csv', delimiter=',', unpack=True)

In [ ]:
fig, ax1 = plt.subplots(figsize=(18,8))
plt.title(r"L-4S-2 First Stage Total Thrust")
plt.ylabel(r"Thrust [kN]")
plt.xlabel(r"Launch Elapsed Time [s]")

plt.plot(stage1_thrust_time, (stage1_thrust*g_0)/1000.0, alpha=0.7, label="L-4S-2 Vehicle Data")

#plt.ylim([0,5])
plt.xlim([0,30])
ax1.legend(loc=1)
plt.show()

We can see the booster thrust die down after about 7 seconds. Looking at the other stages it seems like a good asumption that the thrust curve was roughtly flat. So we'll interpolate the rest of the first stage curve:

In [ ]:
stage1_main_thrust = []
for i, t in enumerate(stage1_thrust_time):
    if t <= 8.0:
        stage1_main_thrust.append(39500)
    else:
        stage1_main_thrust.append(stage1_thrust[i])
    
fig, ax1 = plt.subplots(figsize=(18,8))
plt.title(r"L-4S-2 First Stage Thrust Interpolated")
plt.ylabel(r"Thrust [kN]")
plt.xlabel(r"Launch Elapsed Time [s]")

plt.plot(stage1_thrust_time, (stage1_thrust*g_0)/1000.0, alpha=0.7, label="L-4S-2 Vehicle Data")
plt.plot(stage1_thrust_time, [t*g_0/1000.0 for t in stage1_main_thrust], alpha=0.7, label="Interpolated First Stage Main Motor")

#plt.ylim([0,5])
plt.xlim([0,30])
ax1.legend(loc=1)
plt.show()

And then subtract that curve to get just the booster thrust by itself. There are two boosters, so dividing by two will give the thrust of just one by itself.

In [ ]:
stage1_boost_time = []
stage1_boost_thrust = []


for i, t in enumerate(stage1_thrust_time):
    total = stage1_thrust[i]
    main = stage1_main_thrust[i]
    boost = total - main
    if boost < 0:
        break
    stage1_boost_time.append(t)
    stage1_boost_thrust.append(boost/2.0)
    
    
fig, ax1 = plt.subplots(figsize=(6,8))
plt.title(r"L-4S-2 First Stage Single Booster Thrust")
plt.ylabel(r"Thrust [kN]")
plt.xlabel(r"Launch Elapsed Time [s]")

plt.plot(stage1_boost_time, [t*g_0/1000.0 for t in stage1_boost_thrust], alpha=0.7, label="First Stage Booster Motor")

plt.ylim([0,200])
plt.xlim([0,8.7])
ax1.legend(loc=1)
plt.show()

## Reconstructed Thrust Curve

If we put it all back together, it should exactly match our original thrustcurve

In [ ]:
composit_thrust = []
for i, t in enumerate(stage1_thrust_time):
    if i < len(stage1_boost_thrust):
        composit_thrust.append(stage1_boost_thrust[i]*2.0 + stage1_main_thrust[i])
    else:
        composit_thrust.append(stage1_main_thrust[i])
        
fig, ax1 = plt.subplots(figsize=(18,8))
plt.title(r"L-4S-2 First Composite Thrust")
plt.ylabel(r"Thrust [kN]")
plt.xlabel(r"Launch Elapsed Time [s]")

plt.plot(stage1_thrust_time, (stage1_thrust*g_0)/1000.0, 'k-', lw=3.0, alpha=0.5, label="L-4S-2 Vehicle Data")
plt.plot(stage1_thrust_time, [t*g_0/1000.0 for t in stage1_main_thrust], 'b--', lw=1.0, label="Interpolated First Stage Main Motor")
plt.plot(stage1_boost_time, [t*g_0/1000.0 for t in stage1_boost_thrust], 'g--', lw=1.0, label="First Stage Booster Motor")
plt.plot(stage1_thrust_time, [t*g_0/1000.0 for t in composit_thrust], alpha=1.0, label="Composite First Stage Motor")

#plt.ylim([0,200])
#plt.xlim([0,8.7])
ax1.legend(loc=1)
plt.show()

In [ ]:
# write to file
with open("stage1_main_thrust.csv", 'w') as out:
    for i, t in enumerate(stage1_main_thrust):
        out.write("%0.3f, %0.1f\n" % (stage1_thrust_time[i], t*g_0))

with open("stage1_boost_thrust.csv", 'w') as out:
    for i, t in enumerate(stage1_boost_thrust):
        out.write("%0.3f, %0.1f\n" % (stage1_thrust_time[i], t*g_0))

We can now use both sets of data to create useful thrust curves for simulation.

 - [stage1_main_thrust.csv](stage1_main_thrust.csv)
 - [stage1_boost_thrust.csv](stage1_boost_thrust.csv)

Format:

 Column |                                Value | Units
 :----: | -----------------------------------: | :-----
  1     | Time                                 | s
  2     | Thrust                               | N
